In [1]:
import matplotlib.pyplot as plt
import anndata as adata
import scanpy as sc
import pandas as pd
from harmonypy import run_harmony

import snapatac2 as snap
import anndata as ad
import pandas as pd
import scanpy as sc
import scvi
import numpy as np

scvi.settings.seed = 0
snap.__version__
import warnings

warnings.filterwarnings("ignore")


/home/junyichen/anaconda3/envs/snapatac2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/junyichen/anaconda3/envs/snapatac2/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/junyichen/anaconda3/envs/snapatac2/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_continuous_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
INFO: Seed set to 0
2025-06-03 16:36:07 - INFO - Seed set to 0


In [2]:
import os
default_n_threads = 8
os.environ['OPENBLAS_NUM_THREADS'] = f"{default_n_threads}"
os.environ['MKL_NUM_THREADS'] = f"{default_n_threads}"
os.environ['OMP_NUM_THREADS'] = f"{default_n_threads}"

In [3]:
!export OPENBLAS_NUM_THREADS=8
!export MKL_NUM_THREADS=8
!export OMP_NUM_THREADS=8

# Read single cell data

In [4]:
adata_sc = adata.read_h5ad("/data1st1/hydata/SN_neuron_after_int_filter_anno.h5ad")
# Fix cell type 22 to neuron
# cluster_22_cells = adata_sc.obs_names[adata_sc.obs['Leiden_clusters_res1.0'] == '22']
# adata_sc.obs.loc[cluster_22_cells, 'celltype.L1'] = 'Neuron'

In [5]:
adata_sc = adata_sc[adata_sc.obs.region.isin(['AMY', 'HIP', 'PFC'])]

In [ ]:
adata_sc = adata_sc[adata_sc.obs.region.isin(['AMY', 'HIP', 'PFC'])]

In [27]:
set(adata_sc.obs.level1)

{'Chol', 'Dopa', 'GABA', 'Glut', 'Hist', 'Sero'}

In [22]:
adata_sc.obs['region'].astype(str) + '_' + adata_sc.obs['level1'].astype(str) 

AAACCCAAGAAACACT-1HIP_seekgene_FW32B    HIP_Glut
AAACCCAAGAAACCCA-1HIP_seekgene_FW32B    HIP_Glut
AAACCCAAGAAACCCG-1HIP_seekgene_FW32B    HIP_Glut
AAACCCAAGAAACCTG-1HIP_seekgene_FW32B    HIP_Glut
AAACCCAAGAAACGTC-1HIP_seekgene_FW32B    HIP_Glut
                                          ...   
TTTGTTGCAGCGTAGA-1PFC_novogene_MC33A    PFC_Glut
TTTGTTGCATCCGGTG-1PFC_novogene_MC33A    PFC_Glut
TTTGTTGCATGGAAGC-1PFC_novogene_MC33A    PFC_GABA
TTTGTTGGTGCCTGCA-1PFC_novogene_MC33A    PFC_Glut
TTTGTTGGTGTCCGGT-1PFC_novogene_MC33A    PFC_Glut
Length: 305669, dtype: object

In [42]:
df_marker = pd.read_excel("/data2st1/junyi/output/atac0416/20250423maker list.xlsx")

In [48]:
df_marker.loc[df_marker.region=='PFC','celltype.L2'] = df_marker.loc[df_marker.region=='PFC','celltype.L2'].str.replace('PFC ', '')

In [51]:
adata_sc.obs = adata_sc.obs.merge(df_marker, how='left', left_on='celltype.L2', right_on='celltype.L2')

In [69]:
adata_sc.obs.index = adata_sc.obs.key_0

In [74]:
selected_idx  = adata_sc.obs[~adata_sc.obs['celltype.L2'].isin(['Doublet', 'Cell segment', 'Not sure'])].key_0
adata_sc = adata_sc[selected_idx]

In [79]:
adata_sc.obs.rename(columns={'region_y': 'region'}, inplace=True)

In [83]:
adata_sc.obs.loc[adata_sc.obs.region=='PFC','celltype.L2'] ="PFC_" + adata_sc.obs.loc[adata_sc.obs.region=='PFC','celltype.L2']

In [87]:
adata_sc.obs['celltype.L2'] = adata_sc.obs['celltype.L2'].str.replace('/', '-').str.replace(' ', '_')

In [92]:
adata_sc.obs['region_nt'] = adata_sc.obs['region'].astype(str) + '_' + adata_sc.obs['Neurotransmitter_celltype'].str[:4]

In [99]:
adata_sc.obs['GEX:celltype.L2'] = adata_sc.obs['celltype.L2'].astype(str)

In [104]:
adata_sc = adata_sc[adata_sc.obs.gender=="M"]

In [109]:
for region in adata_sc.obs['region_nt'].unique():
    adata_subset = adata_sc[adata_sc.obs['region_nt'] == region]
    adata_subset.write_h5ad(f"/data1st2/junyi/output/atac0416/subset/region_nt/{region}_sc.h5ad")


... storing 'celltype.L2' as categorical
... storing 'region' as categorical
... storing 'Neurotransmitter_celltype' as categorical
... storing 'celltype.L1' as categorical
... storing 'marker' as categorical
... storing 'region_nt' as categorical
... storing 'GEX:celltype.L2' as categorical
... storing 'celltype.L2' as categorical
... storing 'region' as categorical
... storing 'Neurotransmitter_celltype' as categorical
... storing 'celltype.L1' as categorical
... storing 'marker' as categorical
... storing 'region_nt' as categorical
... storing 'GEX:celltype.L2' as categorical
... storing 'celltype.L2' as categorical
... storing 'region' as categorical
... storing 'Neurotransmitter_celltype' as categorical
... storing 'celltype.L1' as categorical
... storing 'marker' as categorical
... storing 'region_nt' as categorical
... storing 'GEX:celltype.L2' as categorical
... storing 'celltype.L2' as categorical
... storing 'region' as categorical
... storing 'Neurotransmitter_celltype' as c